# ChannelAI 마케팅 AI 모델 설계 (더미 데이터 기반)

이 노트북은 백엔드/플라스크(Flask)에 AI 기능(선형회귀, 이상탐지 등)을 붙이기 전에, **파이썬 환경에서 알고리즘을 먼저 테스트하고 눈으로 확인해보기 위한 실험 공간**입니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# 한글 폰트 설정 (윈도우 환경: 맑은 고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료! (scikit-learn 이 없다면 '!pip install scikit-learn'을 실행하세요)")

## 1. 패턴이 있는 가짜(Dummy) 데이터 생성
단순 무작위 데이터가 아닌, 시간이 지날수록 광고비와 클릭수가 서서히 오르락내리락하는 현실적인 데이터를 30일치 만듭니다.

In [ ]:
# 30일치 날짜 생성
dates = pd.date_range(start="2026-01-01", periods=30)

# 기본 광고비 (점진적으로 우상향하는 패턴 + 약간의 랜덤 노이즈)
base_cost = np.linspace(100000, 200000, 30)
noise = np.random.normal(0, 15000, 30)
cost = np.maximum(base_cost + noise, 0) # 비용이 마이너스가 되지 않게 처리

# 클릭수 (광고비에 비례하되 노이즈 추가)
clicks = cost * 0.05 + np.random.normal(0, 500, 30)

# 특정일에 일부러 '이상치(사고 발생)' 생성 (예: 20일차에 광고비 폭주)
cost[20] = 350000
clicks[20] = 5000 # 비용은 폭주했는데 클릭은 처참함

df = pd.DataFrame({
    'date': dates,
    'cost': cost.astype(int),
    'clicks': clicks.astype(int)
})

df.tail(10) # 생성된 데이터 끝부분 확인

## 2. 선형 회귀 (Linear Regression) 미래 예측
과거 비용(Cost) 추이로 내일(31일차), 모레(32일차) 비용을 예측합니다.

In [ ]:
# 머신러닝을 위해 날짜를 단순 숫자로 차원 변환 (0, 1, 2 ... 29)
X = np.arange(len(df)).reshape(-1, 1)
y = df['cost'].values

# 선형회귀 모델 학습
model = LinearRegression()
model.fit(X, y)

# 향후 3일 예측 (30, 31, 32)
future_X = np.array([[30], [31], [32]])
future_pred = model.predict(future_X)

print("오늘까지의 마지막 비용:", df['cost'].iloc[-1])
print("내일 예상 비용:", int(future_pred[0]))

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(df['date'], df['cost'], label='실제 광고비', marker='o')
plt.plot(pd.date_range(start=df['date'].iloc[-1], periods=4)[1:], future_pred, 
         label='향후 예측', linestyle='--', color='red', marker='x')
plt.title("머신러닝(선형회귀) 기반 예산 소진 예측")
plt.legend()
plt.show()

## 3. 이상 탐지 (Anomaly Detection) - Z-Score 방식
갑자기 돈이 비정상적으로 많이 나간 날(빨간불)을 찾아냅니다.

In [ ]:
# 요약: (현재값 - 평균) / 표준편차
# 값이 2 이상이거나 -2 이하면 '매우 비정상적'인 상태로 판단합니다.
mean_cost = df['cost'].mean()
std_cost = df['cost'].std()

df['z_score'] = (df['cost'] - mean_cost) / std_cost
df['anomaly'] = df['z_score'].apply(lambda x: True if abs(x) > 2 else False)

# 이상치 발생한 날만 뽑아보기
anomalies = df[df['anomaly'] == True]
print("🚨 [경고] 비정상적인 데이터가 감지되었습니다:")
print(anomalies[['date', 'cost']])

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(df['date'], df['cost'], color='blue', alpha=0.5)
plt.scatter(anomalies['date'], anomalies['cost'], color='red', s=100, label='이상 탐지(위험)')
plt.title("비정상적 광고비 지출 알람")
plt.legend()
plt.show()